In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import Counter
import pandas as pd
import json
import logging

logging.basicConfig(level=logging.INFO)


In [ ]:

def load_and_analyze_chars():
    df = pd.read_csv('malayalam_unicode_chars.csv')
    
    byte_patterns = []
    char_to_bytes = {}
    
    for char in df['Character']:
        # Get UTF-8 bytes for each character
        byte_seq = list(char.encode('utf-8'))
        byte_patterns.append(byte_seq)
        char_to_bytes[char] = byte_seq
        
    return df, byte_patterns, char_to_bytes


def get_byte_pair_frequencies(byte_patterns):
    pair_freqs = Counter()
    
    for pattern in byte_patterns:
        for i in range(len(pattern) - 1):
            pair = (pattern[i], pattern[i + 1])
            pair_freqs[pair] += 1
            
    return pair_freqs



In [ ]:

df, byte_patterns, char_to_bytes = load_and_analyze_chars()

print("Sample of characters and their byte patterns:")
for char, bytes_seq in list(char_to_bytes.items())[:5]:
    print(f"Character: {char}, Bytes: {bytes_seq}")


In [ ]:
pair_freqs = get_byte_pair_frequencies(byte_patterns)
print("\nMost common byte pairs:")
for pair, freq in pair_freqs.most_common(5):
    print(f"Pair: {pair}, Frequency: {freq}")

# Additional analysis for understanding the structure
print("\nUnique byte lengths:")
lengths = Counter(len(seq) for seq in byte_patterns)
print(lengths)

print("\nUnique first bytes:")
first_bytes = Counter(pattern[0] for pattern in byte_patterns)
print(sorted(first_bytes.items()))

In [ ]:
def create_byte_pair_merge_rules():
    next_token_id = 256
    merge_rules = {}
    
    # First level merges (first two bytes)
    first_level_merges = {}
    for char, byte_seq in char_to_bytes.items():
        first_pair = (byte_seq[0], byte_seq[1])
        if first_pair not in first_level_merges:
            first_level_merges[first_pair] = next_token_id
            merge_rules[first_pair] = next_token_id
            next_token_id += 1
    
    # Second level merges (merged token + last byte)
    for char, byte_seq in char_to_bytes.items():
        first_pair = (byte_seq[0], byte_seq[1])
        first_token = first_level_merges[first_pair]
        second_pair = (first_token, byte_seq[2])
        merge_rules[second_pair] = next_token_id
        next_token_id += 1
    
    return merge_rules

# Test function
def encode_with_merges(byte_seq, merge_rules):
    tokens = list(byte_seq)
    while len(tokens) > 1:
        merged = False
        for i in range(len(tokens) - 1):
            pair = (tokens[i], tokens[i + 1])
            if pair in merge_rules:
                tokens[i:i+2] = [merge_rules[pair]]
                merged = True
                break
        if not merged:
            break
    return tokens

# Let's test it
merge_rules = create_byte_pair_merge_rules()

# Test a few characters
print("Testing new encoding:")
for char in list(char_to_bytes.keys())[:5]:
    byte_seq = list(char.encode('utf-8'))
    tokens = encode_with_merges(byte_seq, merge_rules)
    print(f"Character: {char}, Bytes: {byte_seq}, Final token: {tokens}")

In [ ]:
merges = list(merge_rules.items())
print(f"Number of merges: {len(merges)}")  # len(merges)

merges[:10]

In [ ]:
from simple_bpe import tokenize, decode_tokens, encode_text, calculate_compression_ratio

In [ ]:
input_text = "മലയാളം"
# input_text = """
# വിക്കിമീഡിയ ഫൗണ്ടേഷന്റെ കീഴിൽ തുളു english ഭാഷയിൽ പ്രവർത്തിക്കുന്ന വിക്കിപീഡിയയാണ് തുളു വിക്കിപീഡിയ. ഇപ്പോൾ 1000-ൽ അധികം ലേഖനങ്ങൾ തുളു വിക്കിപീഡിയയിലുണ്ട്. ഇന്ത്യൻ ഭാഷകളിലെ 23-ആമത്തെ വിക്കിപീഡിയയാണു തുളു വിക്കിപീഡിയ. 8 വർഷത്തെ ഇൻക്യുബേഷനു ശേഷമാണു ഇത് സ്വതന്ത്രമായി പ്രവർത്തനക്ഷമമായത്. 2016-ലെ വിക്കികോൺഫറൻസ് ഇന്ത്യയിൽ വിക്കിമീഡിയ ഫൗണ്ടേഷൻ എക്സിക്യുട്ടീവ് ഡയരക്ടർ കാതറീൻ മെഹർ ആണു തുളു വിക്കിപീഡിയ പ്രഖ്യാപിച്ചത്. 2008 മുതൽ ഇത് ഇൻക്യുബേഷനിൽ ആയിരുന്നു.   തെക്കെപശ്ചിമഘട്ടതദ്ദേശവാസിയായ ഒരു
# """
print(f"Input text: {input_text}")
print(f"Number of characters in input: {len(input_text)}")

raw_tokens = encode_text(input_text)
print(f"Raw byte tokens: {raw_tokens}")
print(f"Number of raw tokens: {len(raw_tokens)}")

input_tokens = tokenize(input_text, merges)
print(f"After applying merges - tokens: {input_tokens}")
print(f"Number of tokens after merges: {len(input_tokens)}")

decoded_text = decode_tokens(input_tokens, merges)
print(f"Decoded text: {decoded_text}")
print(f"Successful roundtrip: {decoded_text == input_text}")

compression_ratio = calculate_compression_ratio(raw_tokens, input_tokens)
print(f"compression ratio : {compression_ratio}")